In [3]:
!nvcc --version
%env OMP_NUM_THREADS=3

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
env: OMP_NUM_THREADS=3


In [4]:
%%writefile sum.cpp

#include <iostream>
#include <vector>
#include <omp.h>
#include <climits>

#include <ctime>
#include <chrono>
#include <cstdlib>

using namespace std;
void generate_random_input(vector<int>& arr, int n)
{
    for (int i = 0; i < n; ++i)
    {
        arr.push_back(rand() % 1000);
    }
    for (int i = 0; i < n; ++i)
    {
      cout<<arr[i]<<" ";
    }
    cout<<endl;
}

void sum_seq(const vector<int>& arr) {
  int sum = 0;
  for (int i = 0; i < arr.size(); i++) {
    sum += arr[i];
  }
  cout << "Sequential Sum: " << sum << endl;
}

void sum_reduction(vector<int>& arr) {
  int sum = 0;
   #pragma omp parallel for reduction(+: sum)
   for (int i = 0; i < arr.size(); i++) {
    sum += arr[i];
  }
  cout << "Sum: " << sum << endl;
}

int main() {
  int n;
  cout << "Enter the number of elements: ";
  cin >> n;

  vector<int> arr;
  generate_random_input(arr, n);

// Parallel reductions
  clock_t start_time = std::clock();
  sum_reduction(arr);
  clock_t end_time = std::clock();
  double parallel_time = double(end_time - start_time) / CLOCKS_PER_SEC;
  cout << "Time taken for parallel reductions: " << parallel_time << " seconds" << endl;
  cout<<endl;

  // Sequential reductions
  start_time = std::clock();
  sum_seq(arr);
  end_time = std::clock();
  double sequential_time = double(end_time - start_time) / CLOCKS_PER_SEC;
  cout << "Time taken for sequential reductions: " << sequential_time << " seconds" << endl;

  return 0;
}

Overwriting sum.cpp


In [5]:
!g++ sum.cpp -o sum -fopenmp

In [7]:
!./sum

Enter the number of elements: 5
383 886 777 915 793 
Sum: 3754
Time taken for parallel reductions: 0.000247 seconds

Sequential Sum: 3754
Time taken for sequential reductions: 2e-06 seconds
